In [1]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [23]:
class ImdbDownloader:
    def __init__(self,link='https://www.imdb.com/search/title/?title_type=feature,tv_movie&view=simple&sort=num_votes,desc'):
        self.request = requests.get(link)
        self.request.encoding = 'UTF-8'
        self.soup = BeautifulSoup(self.request.text)
        self.links = self.PageLinks()
        self.df = FilmDf(self.links).dikt
        
    def PageLinks(self):
        unique_movies = self.soup.find_all('img', {'class':'loadlate'})
        links = ['https://www.imdb.com' + movie.parent['href'] for movie in unique_movies]
        return links
    
    
    
class FilmDf:
    '''
    For the time being only a dictionary containing information on the movies.
    More information will be added and transformed into a Pandas dataframe.
    '''
    def __init__(self,links):
        self.links = links
        self.dikt = self.DataDict()
    
    def getSoup(self, link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text)
    
    def DataDict(self):
        dikt = {'name':[], 'gross_profit':[], 'rating':[]}
        i = 0
        for link in self.links:
            soup = self.getSoup(link)
            if soup.find('div', {'class':'originalTitle'}) is not None:
                dikt['name'].append(soup.find('div', {'class':'originalTitle'}).text)
            else:  
                dikt['name'].append(None)
            if soup.find('h4', text = 'Cumulative Worldwide Gross:') is not None:
                string = re.sub("[^0-9]", "", soup.find('h4', text = 'Cumulative Worldwide Gross:').parent.text)
                dikt['gross_profit'].append(string)
            else:
                dikt['gross_profit'].append(None)
            dikt['rating'].append(soup.find('span', itemprop="ratingValue").text)
        return(dikt)

        
        

In [24]:
ImdbDownloader().df

{'name': ['The Shawshank Redemption (original title)',
  'The Dark Knight (original title)',
  'Inception (original title)',
  'Fight Club (original title)',
  'Pulp Fiction (original title)',
  None,
  'The Matrix (original title)',
  'The Lord of the Rings: The Fellowship of the Ring (original title)',
  'The Lord of the Rings: The Return of the King (original title)',
  'The Godfather (original title)',
  'The Dark Knight Rises (original title)',
  'The Lord of the Rings: The Two Towers (original title)',
  None,
  'Se7en (original title)',
  'Django Unchained (original title)',
  'Gladiator (original title)',
  'Batman Begins (original title)',
  'The Avengers (original title)',
  'The Silence of the Lambs (original title)',
  'Inglourious Basterds (original title)',
  None,
  'Saving Private Ryan (original title)',
  "Schindler's List (original title)",
  'The Departed (original title)',
  'The Prestige (original title)',
  'Star Wars: Episode V - The Empire Strikes Back (original